In [1]:
state = 'IA' 
year = 2020
obj_type = 'inverse_Polsby_Popper'
starting_deviation = 0.01 

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of IA has 4 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from IA_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 18 plans from file.


In [6]:
from pareto import filter_and_sort_pareto
from metrics import scores

plans_scores = [scores(G, plan, G._ideal_population, obj_type) for plan in warm_starts]
_,_,nondominated_warm_starts_plans = filter_and_sort_pareto(plans=warm_starts, upper_bounds=plans_scores, obj_type=obj_type)
print(f"Selected {len(nondominated_warm_starts_plans)} nondominated warm start plans")

Selected 13 nondominated warm start plans


In [ ]:
from epsilon_constraint import epsilon_constraint_method

(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity = 'lcut',                                             # {'lcut', 'scf', 'shir'} 
            cutoff=None,       
            verbose= True,
            warm_start_mode = 'user',                                        # {'None', 'user', 'refinement'}
            warm_starts=warm_starts,                                         # if you have user define warm starts else it is None
            starting_deviation=starting_deviation, 
            time_limit=7200, 
            sizes=None,      
            max_B=True,                                                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True   
            symmetry_breaking='orbitope',                                    # {None, 'orbitope', 'rsum'} 
            state=state,
            year=year
        )

Initially, L = 789617 and U = 805568 and k = 4.

****************************************
Trying deviation = 7975.922500000001
****************************************
Using user-provided warm starts.
Selected warm_start = [[1, 2, 8, 14, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [3, 10, 12, 18, 24, 25, 26, 28, 38, 39, 44, 46, 47, 48, 54, 61, 67, 83, 86, 94, 95, 96, 97, 98], [0, 4, 9, 16, 17, 19, 23, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 49, 50, 52, 53, 55, 57, 58, 62, 63, 64, 66, 70, 73, 74, 81, 89, 91, 93], [5, 6, 7, 11, 13, 15, 29, 34, 45, 51, 56, 69, 72, 75, 77, 80, 82, 85, 87, 88, 90]]
Objective value: 2.124594643540135
Deviation: 663.25

****************************************
Running labeling model!
****************************************
L = 789617 and U = 805568
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1, 1]
Solving the max B problem (as MIP) for use

 15507  1376     cutoff   29         2.02946    1.82782  9.94%   239  435s
 16142  1208     cutoff   27         2.02946    1.84190  9.24%   239  449s
 16780   983    2.00553   25  444    2.02946    1.85767  8.46%   240  461s
 17392   622 infeasible   23         2.02946    1.88263  7.23%   241  474s
 18157     0 infeasible   23         2.02946    1.91100  5.84%   240  483s

Cutting planes:
  Gomory: 24
  Cover: 7
  Clique: 2
  Inf proof: 10
  Zero half: 22
  Mod-K: 2
  RLT: 4
  BQP: 2

Explored 19061 nodes (4488334 simplex iterations) in 485.72 seconds (95.73 work units)
Thread count was 20 (of 20 available processors)

Solution count 2: 2.02946 2.12459 

Optimal solution found (tolerance 0.00e+00)
Best objective 2.029455697163e+00, best bound 2.029455697163e+00, gap 0.0000%

User-callback calls 44232, time in user-callback 0.98 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
plan = [[1, 2, 8, 14, 20, 21, 22

  5896  1857     cutoff   27         2.12459    1.60328  24.5%   245  326s
  6117  1906    1.85055   24  634    2.12459    1.60937  24.3%   244  334s
  6321  1988    1.81995   24  537    2.12459    1.61463  24.0%   245  340s
  6540  2028    2.06796   27  546    2.12459    1.61637  23.9%   245  347s
  6823  2059    2.04950   29  569    2.12459    1.62284  23.6%   245  355s
  7114  2126 infeasible   27         2.12459    1.63426  23.1%   245  364s
  7423  2189    1.86423   25  565    2.12459    1.65170  22.3%   245  368s
  7744  2245    1.97233   23  481    2.12459    1.65187  22.3%   244  375s
  8067  2303    2.09509   23  575    2.12459    1.65871  21.9%   244  381s
  8394  2337     cutoff   23         2.12459    1.66858  21.5%   244  387s
  8682  2368    2.07454   26  567    2.12459    1.67100  21.3%   246  393s
  8967  2405    2.07946   23  444    2.12459    1.68202  20.8%   247  401s
  9244  2446    1.95896   26  561    2.12459    1.68238  20.8%   248  411s
  9651  2481    1.99847  

   585   485    1.26432    7  505    2.12459    1.20746  43.2%   290  107s
   587   486    1.89252   22  491    2.12459    1.20746  43.2%   289  114s
   588   487    1.87457   13  518    2.12459    1.20746  43.2%   288  118s
   589   488    1.52795   10  531    2.12459    1.20746  43.2%   288  120s
   591   489    1.91846   14  532    2.12459    1.20746  43.2%   287  126s
   592   490    1.30209    4  550    2.12459    1.20746  43.2%   286  130s
   594   491    1.23468    7  550    2.12459    1.20746  43.2%   285  137s
   595   492    1.97533   27  551    2.12459    1.20746  43.2%   285  141s
   596   492    1.93081   23  546    2.12459    1.20746  43.2%   284  146s
   597   493    1.27444    8  542    2.12459    1.20746  43.2%   284  150s
   599   494    1.91988   26  562    2.12459    1.20746  43.2%   283  157s
   600   495    1.39831    5  565    2.12459    1.20746  43.2%   282  163s
   601   496    1.51235    9  551    2.12459    1.20746  43.2%   282  165s
   603   497    1.69469  

Selected warm_start = [[1, 2, 8, 14, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [3, 10, 12, 18, 24, 25, 26, 28, 38, 39, 44, 46, 47, 48, 54, 61, 67, 83, 86, 94, 95, 96, 97, 98], [0, 4, 9, 16, 17, 19, 23, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 49, 50, 52, 53, 55, 57, 58, 62, 63, 64, 66, 70, 73, 74, 81, 89, 91, 93], [5, 6, 7, 11, 13, 15, 29, 34, 45, 51, 56, 69, 72, 75, 77, 80, 82, 85, 87, 88, 90]]
Objective value: 2.124594643540135
Deviation: 663.25

****************************************
Running labeling model!
****************************************
L = 794625 and U = 800559
sizes =  [1, 1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Applying warm start!
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (wi

...


CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  7200
FeasibilityTol  1e-07
IntFeasTol  1e-07
MIPGap  0
LazyConstraints  1

Optimize a model with 4759 rows, 4212 columns and 18615 nonzeros
Model fingerprint: 0xfd77c797
Model has 4 quadratic constraints
Variable types: 1200 continuous, 3012 integer (3012 binary)
Coefficient statistics:
  Matrix range     [2e-02, 5e+05]
  QMatrix range    [1e+00, 1e+01]
  Objective range  [3e-01, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+05]

User MIP start produced solution with objective 3.71947 (0.16s)
Loaded user MIP start with objective 3.71947

Presolve removed 2261 rows and 1822 columns
Presolve time: 0.58s
Presolved: 2498 rows, 2390 columns, 11351 nonzeros
Presolved model has 4 quadratic constraint(s)
Variable types: 12 continuous, 2378 integer (2365 binary)

Root rel

 40411 33271    3.37378   65  248    3.71947    1.62258  56.4%   124  647s
 42364 34553    2.54323   30  473    3.71947    1.62335  56.4%   123  663s
 43920 35652    2.18713   30  696    3.71947    1.63322  56.1%   123  677s
 45225 36697    3.10255   71  420    3.71947    1.63322  56.1%   123  694s
 46712 38809    2.85687   33  312    3.71947    1.63322  56.1%   124  712s
 49306 40403     cutoff   63         3.71947    1.64043  55.9%   122  726s
 51317 42196    2.55356   32  560    3.71947    1.65231  55.6%   122  747s
 53496 44256    2.10377   29  563    3.71947    1.65731  55.4%   122  760s
 55925 45076 infeasible   84         3.71947    1.66073  55.4%   121  783s
 57357 46011    2.25355   29  605    3.71947    1.66732  55.2%   121  802s
 58830 48385    2.10176   37  229    3.71947    1.66814  55.2%   122  824s
 61702 50062    2.30703   26  507    3.71947    1.67642  54.9%   121  844s
 63709 51460    3.40431   52  431    3.71947    1.68482  54.7%   120  862s
 65459 52721    2.19099  

 241219 188516    2.28261   23  540    3.71947    1.88272  49.4%   120 2744s
 242862 189499    2.29166   28  480    3.71947    1.88375  49.4%   120 2769s
 244348 191132    2.69679   39  228    3.71947    1.88415  49.3%   120 2786s
 246445 193035    2.38156   28  512    3.71947    1.88577  49.3%   120 2810s
 248792 194095 infeasible   52         3.71947    1.88577  49.3%   120 2830s
 250195 195369    2.11986   26  508    3.71947    1.88781  49.2%   120 2853s
 251854 197133    2.15158   29  524    3.71947    1.88866  49.2%   121 2874s
 253787 197133    2.68071   45  211    3.71947    1.88888  49.2%   120 2875s
 254003 199262    2.59333   34  417    3.71947    1.88999  49.2%   120 2895s
 256634 200933    3.49379   73  384    3.71947    1.89082  49.2%   120 2920s
 258718 202234    3.21126   45  404    3.71947    1.89168  49.1%   120 2940s
 260371 204024    2.13386   23  483    3.71947    1.89287  49.1%   120 2962s
 262692 205512    2.36933   29  634    3.71947    1.89439  49.1%   120 2984s

 435823 337962    3.26262   41  331    3.71947    1.96650  47.1%   119 4800s
 437986 339771    2.86089   43  654    3.71947    1.96752  47.1%   118 4823s
 439840 339771    2.30259   34  416    3.71947    1.96791  47.1%   118 4825s
 440195 340767    2.33622   29  603    3.71947    1.96802  47.1%   118 4848s
 441513 342121    2.75905   37  648    3.71947    1.96870  47.1%   118 4869s
 443185 343658    2.85416   46  307    3.71947    1.96924  47.1%   119 4894s
 444312 343658    3.34073   42  538    3.71947    1.96924  47.1%   119 4895s
 445570 345079    2.40222   40  433    3.71947    1.96943  47.1%   119 4918s
 447451 346490    3.15974   35  567    3.71947    1.96966  47.0%   119 4937s
 449417 347784    2.42818   25  513    3.71947    1.97077  47.0%   119 4960s
 451237 349236    2.35422   35  331    3.71947    1.97138  47.0%   119 4985s
 452990 350795    3.09647   39  582    3.71947    1.97202  47.0%   119 5004s
 454896 352196    2.20429   23  614    3.71947    1.97255  47.0%   119 5021s

 611654 469663    2.26492   24  557    3.71947    2.01329  45.9%   119 6778s


In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
epsilon = 1 / (2 * G._k)
min_deviation = min(round(r[2], 1) for r in result)

if min_deviation < epsilon:
    no_solution_region = None
else:
    no_solution_region = [0, min_deviation]

if no_solution_region is not None:
    print(f"No feasible solution was found within the region: {no_solution_region}")

In [ ]:
from pareto import plot_pareto_frontiers

plot_pareto_frontiers(
                G,
                method='epsilon_constraint_method',
                plans=None,                                   #if method ='epsilon_constraint_method' is None 
                obj_types=obj_type,                               
                ideal_population=G._ideal_population,
                state=state,
                filepath=filepath,
                filename2=filename2,
                no_solution_region=no_solution_region,
                year=year,
                result=result                               #if method ='heuristic' is None 
             )

In [ ]:
from draw import draw_plan
from metrics import observed_deviation_persons, compute_obj

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: int(x)}
G._L = 0
G._U = G._k * G._ideal_population

for plan, obj_bound, dev in result:
    obs_dev = observed_deviation_persons(G, plan, G._ideal_population)
    obj = compute_obj(G, plan, obj_type)
    obj_val = format_obj.get(obj_type, lambda x: round(x, 4))(obj)
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    title = f"{round(obs_dev, 2)}-person deviation ({deviation_percentage}%), with {obj_val} {obj_type}"
    draw_plan(filepath, filename2, G, plan, title=title, year=year)